# Regression task
## Before start

### TODO steps:

1. read csv  

2. write pipeline: split data, train model, predict, evaluate  

3. models: 
    MultiOutputRegressor(LinearRegression()), 
    RandomForestRegressor(), 
    GradientBoostingRegressor() (xgboost), 
    MultiOutputRegressor(SVR()), 
    neural network

4. hyperparameter tuning: 
    GridSearchCV, 
    RandomizedSearchCV

5. dont forget about time series cross validation

## Import libraries

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


## Read data

In [4]:
df_all = pd.read_csv('../data/for_train/train_df.csv')
df_league_0 = pd.read_csv('../data/for_train/df_league_0.csv')
df_league_1 = pd.read_csv('../data/for_train/df_league_1.csv')
df_league_2 = pd.read_csv('../data/for_train/df_league_2.csv')
df_league_3 = pd.read_csv('../data/for_train/df_league_3.csv')
